In [159]:
import numpy as np
import json
import cv2
import os

In [161]:
# Prepare object points for 30mm grid
objp = np.zeros((5*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:5].T.reshape(-1,2)

# objp

In [162]:
# image_folder = "./calibration_data/visible/"
# point_folder = "./extracted_points/visible/"

image_folder = "./calibration_data/thermal_dots_binary/"
point_folder = "./extracted_points/thermal_dots/"

objpoints = []
imgpoints = []
image_size = None

images = os.listdir(image_folder)
for fname in images:
	# Generate image points
	gray = cv2.imread(os.path.join(image_folder, fname), cv2.IMREAD_GRAYSCALE)
	# gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

	# Set blob detector parameters
	params = cv2.SimpleBlobDetector_Params()
	params.filterByCircularity = False
	params.filterByConvexity = False
	params.filterByInertia = False
	params.filterByArea = True
	params.minArea = 4

	# Create a detector with the parameters
	ver = (cv2.__version__).split('.')
	if int(ver[0]) < 3 : detector = cv2.SimpleBlobDetector(params)
	else : detector = cv2.SimpleBlobDetector_create(params)

	ret, corners = cv2.findCirclesGrid(gray, (7, 5), None, blobDetector=detector)
	# ret, corners = cv2.findChessboardCorners(gray, (5,7), None)
	print(ret)
	# ret = False

	# Draw corners
	out = cv2.drawChessboardCorners(cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR), (7, 5), corners, ret)
	cv2.namedWindow("img", cv2.WINDOW_NORMAL)
	cv2.imshow("img", out)
	while cv2.waitKey(0) != ord('q'): continue
	cv2.destroyAllWindows()

	# Save image points
	if ret:
		criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
		# points = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
		points = corners
		
	else:
		name = fname.split(".")[0]
		points = np.loadtxt(os.path.join(point_folder, f"{name}.csv"), delimiter=",", dtype=float)
		points = np.array([np.array(p) for p in points]).astype('float32')

	imgpoints.append(points)

	# Save object points
	objpoints.append(objp)

True
True
True
True
True
True
True
True
True
True


In [163]:
# Compute parameters
vals = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
keys = ("ret", "mtx", "dist", "rvecs", "tvecs")

params = dict(zip(keys, vals))

In [164]:
# Compute new camera matrix
h, w = gray.shape[:2]
vals = cv2.getOptimalNewCameraMatrix(params["mtx"], params["dist"], (w,h), 1, (w,h))
keys = ("newcameramtx", "roi")

params.update(dict(zip(keys, vals)))
params

{'ret': 0.47061607556591145,
 'mtx': array([[106.81900551,   0.        ,  80.18608746],
        [  0.        , 106.59690739,  58.75167093],
        [  0.        ,   0.        ,   1.        ]]),
 'dist': array([[-4.12178322e-01,  2.54950779e-01,  1.19979067e-04,
          6.58896245e-04, -1.22605020e-01]]),
 'rvecs': (array([[ 0.0619533 ],
         [ 0.37447569],
         [-0.09555566]]),
  array([[ 0.16983416],
         [-0.20026203],
         [-0.09054098]]),
  array([[-0.2537458 ],
         [ 0.35591091],
         [-0.01178317]]),
  array([[ 0.09622055],
         [ 0.23957034],
         [-0.10128528]]),
  array([[ 0.15529845],
         [-0.08555605],
         [-0.01720071]]),
  array([[ 0.10602029],
         [ 0.17875406],
         [-0.13271203]]),
  array([[ 0.04736322],
         [ 0.27206284],
         [-0.14399828]]),
  array([[ 0.12870746],
         [ 0.28159082],
         [-0.14210873]]),
  array([[ 0.131383  ],
         [-0.09457398],
         [-0.28153506]]),
  array([[ 0.1706

In [172]:
# Test Undistort
cv2.namedWindow("img", cv2.WINDOW_NORMAL)

images = os.listdir(image_folder)
for fname in images:
	# Load image
	img = cv2.imread(os.path.join(image_folder, fname), cv2.IMREAD_UNCHANGED)

	params["newcameramtx"] = np.array([
		[70, 0.0, 81.92717558174809],
        [0.0, 68, 56.23740168870427], 
        [0.0, 0.0, 1.0]
	])

	# Undistort
	img = cv2.undistort(img, params["mtx"], params["dist"], None, params["newcameramtx"])

	# Crop the image
	# x, y, w, h = params["roi"]
	# img = img[y:y+h, x:x+w]

	# Show image
	cv2.imshow("img", img)
	while cv2.waitKey(0) != ord('q'): continue

cv2.destroyAllWindows()

In [175]:
class LeptonDewarp:
    camera_matrix = np.array([
        [104.65403680863373, 0.0, 79.12313258957062],
        [0.0, 104.48251047202757, 55.689070170705634],
        [0.0, 0.0, 1.0]
    ])

    distortion_coeffs = np.array([
        -0.397583085816071270,
        +0.180686417456711930,
        +0.004626461618389028,
        +0.004197358204037882,
        -0.033813994995914630
    ])

    new_cam_matrix = np.array([
        [66.54581451416016, 0.0, 81.92717558174809],
        [0.0, 64.58526611328125, 56.23740168870427], 
        [0.0, 0.0, 1.0]
    ])

    def dewarp(self, img, crop=True):
        out = cv2.undistort(
            src=img, 
            cameraMatrix=self.camera_matrix,
            distCoeffs=self.distortion_coeffs,
            newCameraMatrix=self.new_cam_matrix
        )

        if crop:
            row, col = out.shape[0:2]
            return out[16:row-16, 12:col-12]
        else: return out
        
# Test Undistort
cv2.namedWindow("img", cv2.WINDOW_NORMAL)

dw = LeptonDewarp()

images = os.listdir(image_folder)
for fname in images:
	# Load image
	img = cv2.imread(os.path.join(image_folder, fname), cv2.IMREAD_UNCHANGED)

	# Undistort
	img = dw.dewarp(img)

	# Crop the image
	# x, y, w, h = params["roi"]
	# img = img[y:y+h, x:x+w]

	# Show image
	cv2.imshow("img", img)
	while cv2.waitKey(0) != ord('q'): continue

cv2.destroyAllWindows()

In [38]:
# Check if object is iterable
def is_iter(obj):
	try: iter(obj)
	except: return False
	return True

# Convert iterables to lists
def recursive_convert(some_iter):
	# Convert parent elements
	if type(some_iter) == np.ndarray:
		some_iter = some_iter.tolist()
	else: some_iter = list(some_iter)

	# Convert child elements
	for idx, elem in enumerate(some_iter):
		if is_iter(elem):
			some_iter[idx] = recursive_convert(elem)
	
	return some_iter

# Convert numpy arrays to python arrays
for k in params:
	if is_iter(params[k]):
		params[k] = recursive_convert(params[k])

print(params)

# Save calibration parameters to JSON
with open("visible_params.json", "w") as f:
	json.dump(params, f, indent="\t")
	f.close()

{'ret': 0.18843763406338257, 'mtx': [[524.2755675157524, 0.0, 322.5883217167881], [0.0, 525.772708925735, 242.7950238884331], [0.0, 0.0, 1.0]], 'dist': [[0.21278355925005868, -0.47699693160945694, -0.0011051479542858727, -0.0012412176630451593, 0.2923106324942272]], 'rvecs': [[[-0.2475632335809654], [-0.07118388910618222], [0.018128024362096543]], [[-0.24655066637714632], [-0.08933915500551615], [0.01965856942833153]], [[-0.24373241778761864], [-0.01729020765880648], [0.010753564399575085]], [[-0.07351324368951012], [0.011611545009801091], [0.0031926211405833905]], [[-0.07680293484436597], [0.030179515108039946], [-0.0011318604082684674]], [[-0.07630727649406926], [0.047402661429152344], [-0.0008382545697365868]], [[-0.07209540196156447], [-0.017463687592352253], [0.009745002981140538]], [[-0.12185504943449557], [-0.22126318728830618], [0.042182389484873906]], [[-0.08825097571798324], [0.044584993255638576], [-0.045342701699483214]]], 'tvecs': [[[-0.092251039146032], [-0.05034086279878